In [1]:
import numpy as np
import time
import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import fetch_mldata
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_random_state
from sklearn.utils.extmath import safe_sparse_dot, squared_norm
from scipy.misc import comb, logsumexp 
from sklearn.linear_model import SGDClassifier

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import random
from scipy.optimize import minimize

ImportError: No module named model_selection

In [ ]:
mnist = fetch_mldata('MNIST original')
X = mnist.data.astype('float64')
y = mnist.target


train_samples = 30000

random_state = check_random_state(0)

permutation = random_state.permutation(X.shape[0]) 

X = X[permutation]
y = y[permutation]
X = X.reshape((X.shape[0], -1))

X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=train_samples, test_size=10000, random_state=1)

#Normalize Data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Y_train = np.zeros((len(y_train), 10))
for i,j in enumerate(y_train):
    Y_train[i, int(j)] = 1
Y_test = np.zeros((len(y_test), 10))
for i,j in enumerate(y_test):
    Y_test[i, int(j)] = 1

In [ ]:
#Defining a loss function
def loss_function(X,Y, w, alpha=0):
    n_classes = Y.shape[1]
    n_features = X.shape[1]
    w = w.reshape(n_classes, -1)
    fit_intercept = w.size == (n_classes * (n_features + 1))
    old_w = w.copy()
    if fit_intercept:
        intercept = w[:, -1]
        w = w[:, :-1]
    else:
            intercept = 0
    p = safe_sparse_dot(X, w.T)
    p += intercept
    p -= logsumexp(p, axis=1)[:, np.newaxis]
    loss = -(Y * p).sum()
    loss += 0.5 * alpha * squared_norm(w)
    p = np.exp(p, p)
    return loss, p, w

In [ ]:
def grad_loss(X,Y, w, alpha=0):
    # X shape: n_sample*n_feature
    # Y shape: n_sample*n_class indicator matrix
    # w shape: n_class*n_feature
    n_classes = Y.shape[1]
    n_features = X.shape[1]
    w = w.reshape(n_classes, -1)
    fit_intercept = w.size == (n_classes * (n_features + 1))
    old_w = w.copy()
    if fit_intercept:
        intercept = w[:, -1]
        w = w[:, :-1]
    else:
        intercept = 0
            
    b = safe_sparse_dot(X, w.T) # shape: n_sample*n_class
    b += intercept
    p1 = np.exp(b)
    p2 = p1./np.exp(b, axis=1).sum()[:, np.newaxis] #x_i*exp(x_i\dot w_k)/\sum_k'{exp(x_i\dot w_k')}
    #shape: n_sample*n_class 
    grad = safe_sparse_dot(X.T, Y) - safe_sparse_dot(X.T, p2) #shape: n_feature*n_class
    grad += alpha * w.T
    grad = grad.T
    return grad #shape: n_class*n_feature



In [ ]:

def sgd_skl(X_train, Y_train, momentum=0.9, lr=0.01, batch_size=1001, alpha=0.1, maxepoch=50, eps=1e-8):
    
    #Batch
    Index1 = np.arange(X_train.shape[0])
    np.random.shuffle(Index1)
    n_batch = int(X_train.shape[0]/batch_size)
    start_ind = np.arange(n_batch)*batch_size
    end_ind = (np.arange(n_batch)+1)*batch_size+1

    mu = list()
    eta = list()
    w = list()

    mu.append(lr) 
    eta.append(momentum) 
    deltalambda = list()

    #Initialization
    ite = 0 # No. interation
    inds = Index1[start_ind[ite]:end_ind[ite]]
    
    model = list()
    model.append(SGDClassifier(loss='log', penalty="l2", learning_rate='constant', eta0=mu[ite], alpha=eta[ite], epsilon=eps))
    model[ite].fit(X_train[inds], Y_train[inds]) 
    w.append(model[ite].coef_)
    mean_accuracy = list()
    mean_accuracy.append(model[ite].score(X_train[inds], Y_train[inds]))

    #Interations
    #Each interation processes a single batch
    
    for ite in np.arange(n_batch): 

        inds = Index1[start_ind[ite]:end_ind[ite]]

        #Phi(s_t,lam_t)
        model.append(SGDClassifier(loss='log', penalty="l2", learning_rate='optimal', eta0=mu[ite], alpha=eta[ite], epsilon=eps))      
        model[ite+1].fit(X_train[inds], Y_train[inds])
        
        deltaphi = model[ite+1].score(X_train[inds], Y_train[inds]) - model[ite].score(X_train[inds], Y_train[inds])

        w.append(model[ite+1].coef_)
        deltaw = w[ite+1] - w[ite]

        params = model[ite+1].get_params(deep=True) 
        mu.append(params['eta0'])
        eta.append(params['alpha'])
        deltalambda.append([mu[ite+1] - mu[ite], eta[ite + 1] - eta[ite]])
        
        #print state_t[ite+1].get_params(deep=True)['alpha']
        #print state_t[ite+1].get_params(deep=True)['eta0']

        mean_accuracy.append(clf[ite+1].score(X_train[inds], Y_train[inds]))
        print(mean_accuracy[ite+1])
        error = 1-mean_accuracy[ite]
        v[ite+1] = mu[ite]*v[ite] + grad_J(w[ite+1], w[ite])
        w[ite+1] = w[ite] - eta[ite]*(mu[ite]*v[ite] - grad_J(w[ite+1], w[ite]))

    return mu, eta, w, state_t, mean_accuracy


out = sgd_skl(X_train, y_train)